In [1]:
PATH_IMAGES = "./data/imgs/"
PATH_MODELS = "./models/"

EPOCHS = 80
BATCH_SIZE = 60

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

if not(os.path.isdir(PATH_MODELS)):
    os.mkdir(PATH_MODELS)

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

#import tensorflow_lattice as tfl
print(tf.__version__)
print(tf.keras.backend.image_data_format())
import scipy.io 

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import cv2
from PIL import Image

import random
import sys
import math

from datetime import datetime

#tf.debugging.set_log_device_placement(True)

def pinball_loss(y_true, y_pred, tau=.5):
    """Computes the pinball loss between `y_true` and `y_pred`.
    `loss = maximum(tau * (y_true - y_pred), (tau - 1) * (y_true - y_pred))`
    In the context of regression this, loss yields an estimator of the tau
    conditional quantile.
    See: https://en.wikipedia.org/wiki/Quantile_regression
    Usage:
    ```python
    loss = pinball_loss([0., 0., 1., 1.], [1., 1., 1., 0.], tau=.1)
    # loss = max(0.1 * (y_true - y_pred), (0.1 - 1) * (y_true - y_pred))
    #      = (0.9 + 0.9 + 0 + 0.1) / 4
    print('Loss: ', loss.numpy())  # Loss: 0.475
    ```
    Args:
      y_true: Ground truth values. shape = `[batch_size, d0, .. dN]`
      y_pred: The predicted values. shape = `[batch_size, d0, .. dN]`
      tau: (Optional) Float in [0, 1] or a tensor taking values in [0, 1] and
        shape = `[d0,..., dn]`.  It defines the slope of the pinball loss. In
        the context of quantile regression, the value of tau determines the
        conditional quantile level. When tau = 0.5, this amounts to l1
        regression, an estimator of the conditional median (0.5 quantile).
    Returns:
        pinball_loss: 1-D float `Tensor` with shape [batch_size].
    References:
      - https://en.wikipedia.org/wiki/Quantile_regression
      - https://projecteuclid.org/download/pdfview_1/euclid.bj/1297173840
    """
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    # broadcast the pinball slope along the batch dimension, and clip to
    # acceptable values
    tau = tf.expand_dims(tf.cast(tau, y_pred.dtype), 0)
    one = tf.cast(1, tau.dtype)

    delta_y = y_true - y_pred
    pinball = tf.math.maximum(tau * delta_y, (tau - one) * delta_y)
    return tf.reduce_mean(tf.keras.backend.batch_flatten(pinball), axis=-1)


class PinballLoss(tf.keras.losses.Loss):
    """Computes the pinball loss between `y_true` and `y_pred`.
    `loss = maximum(tau * (y_true - y_pred), (tau - 1) * (y_true - y_pred))`
    In the context of regression, this loss yields an estimator of the tau
    conditional quantile.
    See: https://en.wikipedia.org/wiki/Quantile_regression
    Usage:
    ```python
    pinball = tfa.losses.PinballLoss(tau=.1)
    loss = pinball([0., 0., 1., 1.], [1., 1., 1., 0.])
    # loss = max(0.1 * (y_true - y_pred), (0.1 - 1) * (y_true - y_pred))
    #      = (0.9 + 0.9 + 0 + 0.1) / 4
    print('Loss: ', loss.numpy())  # Loss: 0.475
    ```
    Usage with the `compile` API:
    ```python
    model = tf.keras.Model(inputs, outputs)
    model.compile('sgd', loss=tfa.losses.PinballLoss(tau=.1))
    ```
    Args:
      tau: (Optional) Float in [0, 1] or a tensor taking values in [0, 1] and
        shape = `[d0,..., dn]`.  It defines the slope of the pinball loss. In
        the context of quantile regression, the value of tau determines the
        conditional quantile level. When tau = 0.5, this amounts to l1
        regression, an estimator of the conditional median (0.5 quantile).
      reduction: (Optional) Type of `tf.keras.losses.Reduction` to apply to
        loss. Default value is `AUTO`. `AUTO` indicates that the reduction
        option will be determined by the usage context. For almost all cases
        this defaults to `SUM_OVER_BATCH_SIZE`.
        When used with `tf.distribute.Strategy`, outside of built-in training
        loops such as `tf.keras` `compile` and `fit`, using `AUTO` or
        `SUM_OVER_BATCH_SIZE` will raise an error. Please see
        https://www.tensorflow.org/alpha/tutorials/distribute/training_loops
        for more details on this.
      name: Optional name for the op.
    References:
      - https://en.wikipedia.org/wiki/Quantile_regression
      - https://projecteuclid.org/download/pdfview_1/euclid.bj/1297173840
    """

    def __init__(self,
                 tau=.5,
                 reduction=tf.keras.losses.Reduction.AUTO,
                 name='pinball_loss'):
        super(PinballLoss, self).__init__(reduction=reduction, name=name)
        self.tau = tau

    def call(self, y_true, y_pred):
        return pinball_loss(y_true, y_pred, self.tau)

    def get_config(self):
        config = {
            'tau': self.tau,
        }
        base_config = super(PinballLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

2 Physical GPUs, 2 Logical GPUs
2.0.0
channels_last


In [3]:
df = pd.read_pickle("./data/metadata_small.pkl")

In [4]:
df.Split.value_counts()

train     126928
unused     27653
test       25969
val        17038
Name: Split, dtype: int64

In [5]:
# Random test image
if False:
    fig, ax = plt.subplots(3,5, figsize=(18,8))
    for i in range (3):
        for j in range (5):
            path = df[(df.PersonIdentity == 0) & (df.Recording == 1)].sample().iloc[0].Path
            img = Image.open(path)
            ax[i][j].imshow(img)

In [6]:
def make_dataset(df):
    fileNames = df.Path.to_list()
    gazeDirs = df.Gaze.to_list()
    images = []
    for fileName, gazeDirs in zip(fileNames, gazeDirs):
        frame_number = int(fileName.split('/')[-1][:-4])
        lists_sources = []
        for j in range(-3,4):
            name_frame = '/'.join(fileName.split('/')[:-1]+['%0.6d.jpg'%(frame_number+j)])
            lists_sources.append(name_frame)
            
        item = (lists_sources,gazeDirs)
        images.append(item)
    return images

def loader(path):
    try:
        #image = tf.io.read_file(path)
        #image = tf.image.decode_jpeg(image)
        #image = tf.image.resize(image, [224,224])
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224,224))
        return image
    except Exception:
        print("Error:", path)
        return np.zeros((224,224,3))
        
class GazeDataset(tf.data.Dataset):
    def _generator(split, num_samples):
        df = pd.read_pickle("./data/metadata_small.pkl")
        df = df[df.Split == split.decode()]
        data = make_dataset(df)
        while True:
            random.shuffle(data)
            for idx in range(0, len(data)-num_samples, num_samples):
                lstImg = []
                lstGaze = []
                for i in range(num_samples):
                    path_source, gaze = data[idx+i]
                    # Reading data (line, record) from the file
                    imgs = []
                    for i, frame_path in enumerate(path_source):
                        img = loader(frame_path)
                        imgs.append(img)
                    lstGaze.append(gaze)
                    lstImg.append(np.stack(imgs))  
                    #print(tf.shape(imgs))

                yield np.stack(lstImg), np.stack(lstGaze)
                #yield np.array(lstImg), np.array(lstGaze)
    
    def __new__(cls, split, num_samples):
        return tf.data.Dataset.from_generator(
            cls._generator,
            output_types=(tf.float32, tf.float32),
            #=tf.float32,
            output_shapes=([None, 7, 224,224,3], [None, 2]),
            args=(split, num_samples,)
        )

In [7]:
import time
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        i = 0
        for sample in dataset:
            # Performing a training step
            print(datetime.now(), epoch_num, i)
            i = i + 1
            break
    tf.print("Execution time:", time.perf_counter() - start_time)
    
benchmark(GazeDataset("train", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE))

2020-04-04 17:19:50.579977 0 0
2020-04-04 17:19:53.726773 1 0
Execution time: 6.657734099775553


In [8]:
for sample in GazeDataset("train", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE):
    # Performing a training step
    print(datetime.now(), "x", "i")
    break

2020-04-04 17:19:57.198906 x i


In [9]:
sample[0].shape

TensorShape([60, 7, 224, 224, 3])

In [10]:
sample[1].shape

TensorShape([60, 2])

In [11]:
def error_Yaw(y_pred, y_true):
    error = tf.math.abs(y_true[:,0] - y_pred[:,0])
    error = 180.0 * tf.math.reduce_mean(error) / math.pi
    return error

def error_Pitch(y_pred, y_true):
    error = tf.math.abs(y_true[:,1] - y_pred[:,1])
    error = 180.0 * tf.math.reduce_mean(error) / math.pi
    return error


In [ ]:
time = datetime.now()
timeStr = str(time).split(".")[0].replace(" ", "-").replace(":", "-")

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    resnet = tf.keras.applications.MobileNetV2()
    resnet.trainable = False

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input((7, 224, 224, 3)),
        tf.keras.layers.TimeDistributed(resnet),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256)),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 128, return_sequences = True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 128, return_sequences = True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units = 128)),
        tf.keras.layers.Dense(2)
    ])
    model.summary()


    model.compile(optimizer='adam', loss=PinballLoss(), metrics=[error_Pitch, error_Yaw])



    path = "%scrf.%s-gaze360{epoch:03d}-{val_loss:.4f}.hdf5." % (PATH_MODELS, time)
    callbackSaver = tf.keras.callbacks.ModelCheckpoint(path, monitor='val_loss')
    #model.fit_generator will be deprecated use fit instead
    model.fit(GazeDataset("train", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE),
                        steps_per_epoch=int(len(df[df.Split == "train"])/BATCH_SIZE),
                        epochs=EPOCHS,
                        validation_data=GazeDataset("val", BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE),
                        validation_steps=int(len(df[df.Split == "val"])/BATCH_SIZE),
                        callbacks=[])
                       #workers=30,
                       #use_multiprocessing=False)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 7, 1000)           3538984   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 256)            256256    
_________________________________________________________________
bidirectional (Bidirectional (None, 7, 256)            394240    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 4,584,234
Trainable params: 1,045,250
Non-trainable params: 3,538,984
_________________________________________________________________
Train for 2115 steps, validate for 283 steps
Epoch 1/80
2115/2115 [==============================] - 225